# **CAP 5404 Deep Learning for Computer Graphics**
# *Project II. Neural Networks & Computer Graphics*

Pranath Reddy Kumbam (**UFID**: 8512-0977)


## CPU vs GPU

### Load Datasets

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Path to Working Directory 
%cd drive/My Drive/Acad/DLCG/Project2

/content/drive/My Drive/Acad/DLCG/Project2


In [ ]:
# Import libraries
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import os
import random
import time
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

In [ ]:
# Mean Chrominance values
def get_mean_chrominance(img1, img2):
  return (np.mean(img1), np.mean(img2))

# Min-Max Norm 
def norm(images):
    data = []
    for sample in images:
      min = np.amin(sample)
      max = np.amax(sample)
      range_val = max - min
      sample = (sample-min)/range_val
      sample = sample.reshape(1,128,128)
      data.append(sample)
    data = np.asarray(data)
    return data

# Import Data
l_train = norm(np.load('./Data/arrays/Faces/L_train.npy'))
a_train = norm(np.load('./Data/arrays/Faces/a_train.npy'))
b_train = norm(np.load('./Data/arrays/Faces/b_train.npy'))

x_train = l_train
y_train = np.array([get_mean_chrominance(a_train[x], b_train[x]) for x in range(x_train.shape[0])])

batch_size = 100
# Shuffle Data
x_train, y_train = shuffle(x_train, y_train, random_state=0)

# Split into batches
batch_size = 100
a = 0
b = batch_size
data_temp = []
data_temp2 = []
for i in range(int(x_train.shape[0]/batch_size)):
    data_temp.append(x_train[a:b])
    data_temp2.append(y_train[a:b])
    a += batch_size
    b += batch_size
x_train = np.asarray(data_temp)
y_train = np.asarray(data_temp2)

# Print data shape
print("Data Shape")
print(x_train.shape)
print(y_train.shape)

Data Shape
(60, 100, 1, 128, 128)
(60, 100, 2)


### Define Model

In [ ]:
# As described in the project description 
# A simple CNN model with seven Conv Blocks
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()

        self.regressor = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(16, 32, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(128, 256, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(256, 64),
            nn.Linear(64, 2),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.regressor(x)
        return x

model_cpu = CNN().to("cpu") # Push model to cpu
model_gpu = CNN().to("cuda") # Push model to gpu

### Train CPU Model

In [ ]:
# Loss Function
criteria = torch.nn.MSELoss()

# Optimizer
optimizer = torch.optim.Adam(model_cpu.parameters(), lr=2e-3, weight_decay=1e-5)
n_epochs = 30

In [ ]:
# Training
loss_array = []
start = time.time()
pbar = tqdm(range(1, n_epochs+1))
for epoch in pbar:
    train_loss = 0.0
    
    for i in range(x_train.shape[0]):

        data = torch.from_numpy(x_train[i].astype('float32'))
        labels = torch.tensor(y_train[i], dtype=torch.float, device="cpu")
        optimizer.zero_grad()
        outputs = model_cpu(data)
        loss = criteria(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss

    train_loss_avg = train_loss/x_train.shape[0]
    pbar.set_postfix({ 'Training Loss': train_loss_avg.detach().cpu().numpy() })  
end = time.time()
cpu_runtime = end - start # Get CPU runtime

  0%|          | 0/30 [00:00<?, ?it/s]

### Train GPU Model

In [ ]:
# Training
loss_array = []
start = time.time()
pbar = tqdm(range(1, n_epochs+1))
for epoch in pbar:
    train_loss = 0.0
    
    for i in range(x_train.shape[0]):

        data = torch.from_numpy(x_train[i].astype('float32'))
        if torch.cuda.is_available():
          data = data.cuda()
        labels = torch.tensor(y_train[i], dtype=torch.float, device="cuda")
        optimizer.zero_grad()
        outputs = model_gpu(data)
        loss = criteria(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss

    train_loss_avg = train_loss/x_train.shape[0]
    pbar.set_postfix({ 'Training Loss': train_loss_avg.detach().cpu().numpy() })  
end = time.time()
gpu_runtime = end - start # Get GPU runtime

  0%|          | 0/30 [00:00<?, ?it/s]

### Calculate Speed-Up

In [ ]:
ratio = cpu_runtime/gpu_runtime # Calculate Speedup
print("The speedup offered by moving the model to GPU: " + str(ratio))

The speedup offered by moving the model to GPU: 8.505974659056736
